In [154]:
import pandas as pd

In [155]:
# Load the data
pd_excel_file = pd.ExcelFile("../data/landing/historical_data.xlsx")

In [156]:
processed_data = []

In [157]:
def parse_sheet(df_sheet):
    # sheet_0 = pd_excel_file.parse(0, header=None)
    df_sheet[0] = df_sheet[0].ffill()
    
    # forward fill the first row
    df = df_sheet.ffill(axis=1)
    
    print(df.head())
    return df

In [158]:
new_df = []

In [159]:
for sheet in range(len(pd_excel_file.sheet_names)):
    df_sheet = pd_excel_file.parse(sheet, header=[0, 1, 2])
    suburb_names = df_sheet.iloc[:, 1]
    for column in df_sheet.iloc[:, 2:]:
        for item in range(0, len(df_sheet[column])):
            new_df.append([pd_excel_file.sheet_names[sheet], column, suburb_names[item], df_sheet[column][item]])

In [160]:
new_df = pd.DataFrame(new_df)
new_df

,0,1,2,3
0,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Albert Park-Middle Park-West St Kilda,352
1,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Armadale,210
2,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Carlton North,87
3,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Carlton-Parkville,303
4,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",CBD-St Kilda Rd,755
...,...,...,...,...
207013,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Wanagaratta,380
207014,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Warragul,440
207015,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Warrnambool,420
207016,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Wodonga,410


In [161]:
new_df.columns = ["type", "info", "suburb", "value"]

In [162]:
# Store the number of splits made
new_df[['Lease Info', 'Date', 'Count Median']] = pd.DataFrame(new_df['info'].tolist(), index=new_df.index)
new_df

,type,info,suburb,value,Lease Info,Date,Count Median
0,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Albert Park-Middle Park-West St Kilda,352,Lease commenced in year ending,Mar 2000,Count
1,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Armadale,210,Lease commenced in year ending,Mar 2000,Count
2,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Carlton North,87,Lease commenced in year ending,Mar 2000,Count
3,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",Carlton-Parkville,303,Lease commenced in year ending,Mar 2000,Count
4,1 bedroom flat,"(Lease commenced in year ending , Mar 2000, Co...",CBD-St Kilda Rd,755,Lease commenced in year ending,Mar 2000,Count
...,...,...,...,...,...,...,...
207013,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Wanagaratta,380,Lease commenced in year ending,Mar 2023,Median
207014,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Warragul,440,Lease commenced in year ending,Mar 2023,Median
207015,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Warrnambool,420,Lease commenced in year ending,Mar 2023,Median
207016,All properties,"(Lease commenced in year ending , Mar 2023, Me...",Wodonga,410,Lease commenced in year ending,Mar 2023,Median


In [163]:
# Split the df into two based on Count Median
new_df_1 = new_df[new_df['Count Median'] == 'Count'].rename(columns={'value': 'Count'})
new_df_2 = new_df[new_df['Count Median'] == 'Median'].rename(columns={'value': 'Median'})

In [164]:
# Merge the two dataframes based on the type, suburb and date
new_df_1 = new_df_1.merge(new_df_2[['type', 'suburb', 'Date', "Median"]], on=['type', 'suburb', 'Date'])

In [165]:
new_df_1.drop(columns=['info', 'Count Median', 'Lease Info'], inplace=True)
new_df_1

,type,suburb,Count,Date,Median
0,1 bedroom flat,Albert Park-Middle Park-West St Kilda,352,Mar 2000,165
1,1 bedroom flat,Armadale,210,Mar 2000,150
2,1 bedroom flat,Carlton North,87,Mar 2000,150
3,1 bedroom flat,Carlton-Parkville,303,Mar 2000,165
4,1 bedroom flat,CBD-St Kilda Rd,755,Mar 2000,250
...,...,...,...,...,...
202855,All properties,Group Total,11944,Mar 2023,485
202856,All properties,Group Total,11944,Mar 2023,425
202857,All properties,Group Total,11944,Mar 2023,380
202858,All properties,Group Total,11944,Mar 2023,400


In [166]:
# Set "-" to NaN
new_df_1 = new_df_1.replace("-", pd.NA)

In [167]:
# Save the data
new_df_1.to_csv("../data/raw/historical_data.csv", index=False)